In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from sklearn.ensemble import RandomForestClassifier


In [ ]:
df_tr_01 = pd.read_csv('training_data.csv')
print(len(df_tr_01))
print(df_tr_01.head())

In [ ]:
df_unte_01 = pd.read_csv('unlabelled_test_data.csv')
print(len(df_unte_01))
print(df_unte_01.head())

In [ ]:
print(df_tr_01.shape)
print(df_tr_01.info())
print('----------')
df_tr_02 = df_tr_01.copy()
for i in range(0, len(df_tr_02.difficulty)):
    if df_tr_02.iloc[i,2] == "A1":
        df_tr_02.iloc[i,2] = int(0)
    elif df_tr_02.iloc[i,2] == "A2":
        df_tr_02.iloc[i,2] =int(1)
    elif df_tr_02.iloc[i,2] == "B1":
        df_tr_02.iloc[i,2] = int(2)
    elif df_tr_02.iloc[i,2] == "B2":
        df_tr_02.iloc[i,2] = int(3)
    elif df_tr_02.iloc[i,2] == "C1":
        df_tr_02.iloc[i,2] = int(4)
    elif df_tr_02.iloc[i,2] == "C2":
        df_tr_02.iloc[i,2] = int(5)
df_tr_02.difficulty.unique()

df_tr_05_02 = df_tr_01.copy()
for i in range(0, len(df_tr_02.difficulty)):
    if df_tr_05_02.iloc[i,2] == "A1":
        df_tr_05_02.iloc[i,2] = int(0)
    elif df_tr_05_02.iloc[i,2] == "A2":
        df_tr_05_02.iloc[i,2] =int(1)
    elif df_tr_05_02.iloc[i,2] == "B1":
        df_tr_05_02.iloc[i,2] = int(2)
    elif df_tr_05_02.iloc[i,2] == "B2":
        df_tr_05_02.iloc[i,2] = int(3)
    elif df_tr_05_02.iloc[i,2] == "C1":
        df_tr_05_02.iloc[i,2] = int(4)
    elif df_tr_05_02.iloc[i,2] == "C2":
        df_tr_05_02.iloc[i,2] = int(5)
df_tr_05_02.difficulty.unique()


In [ ]:
print(df_tr_02.head())

In [ ]:
classes = df_tr_02.sort_values(by=['difficulty'], ascending=True).difficulty.unique()
print(classes)
print(len(df_tr_02))
counts = []
for i in classes:
    count = len(df_tr_02[df_tr_02['difficulty'] == i])
    counts.append(count)
plt.bar(['A1', 'A2', 'B1', 'B2', 'C1', 'C2'], counts)
plt.show()
    

In [ ]:
x = df_tr_02.sentence.values.tolist()
y = df_tr_02.difficulty.values.tolist()

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=32)

In [ ]:
print(y)

In [ ]:
#x_train.shape
type(x_train)

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)

x_vect_train = vectorizer.transform(x_train)
x_vect_test = vectorizer.transform(x_test)

print(x_vect_train[0])

In [ ]:
import re

d = ",.!?/&-:;@'..."
"["+"\\".join(d)+"]"

s = x_train[0]
s = ' '.join(w for w in re.split(r"\W", s) if w)

for i in s.split():
    if len(i)> 1: print(i, vectorizer.vocabulary_[i.lower()])

# Logistic regression

In [ ]:
classifier = LogisticRegression(max_iter=1000)
classifier.fit(x_vect_train, y_train)

In [ ]:
score = classifier.score(x_vect_test, y_test)
print('Accuracy', score)

In [ ]:
y_pred = classifier.predict(x_vect_test)

difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in y_pred]
original_difficulties = [difficulty_levels[label] for label in y_test]

cm = confusion_matrix(original_difficulties, predicted_difficulties, labels=["A1","A2","B1","B2","C1","C2"])
df_cm = pd.DataFrame(cm, index=["A1","A2","B1","B2","C1","C2"], columns= ["A1","A2","B1","B2","C1","C2"])
df_cm

In [ ]:
sb.heatmap(cm.T, square=True, annot=True, fmt='d', cbar=False, xticklabels=["A1","A2","B1","B2","C1","C2"], yticklabels=["A1","A2","B1","B2","C1","C2"])

plt.xlabel('true label')
plt.ylabel('predicted label')

# KNN

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)
x_vect_train_scaled = scaler.fit_transform(x_vect_train)
x_vect_test_scaled = scaler.transform(x_vect_test) 

In [ ]:
classifier_knn = KNeighborsClassifier(n_neighbors=2, weights='distance')
classifier_knn.fit(x_vect_train, y_train)

In [ ]:
score_knn = classifier_knn.score(x_vect_test_scaled, y_test)
print('Accuracy', score_knn)

In [ ]:
y_pred = classifier_knn.predict(x_vect_test_scaled)

difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in y_pred]
original_difficulties = [difficulty_levels[label] for label in y_test]

cm = confusion_matrix(original_difficulties, predicted_difficulties, labels=["A1","A2","B1","B2","C1","C2"])
df_cm = pd.DataFrame(cm, index=["A1","A2","B1","B2","C1","C2"], columns= ["A1","A2","B1","B2","C1","C2"])
df_cm

In [ ]:
sb.heatmap(cm.T, square=True, annot=True, fmt='d', cbar=False, xticklabels=["A1","A2","B1","B2","C1","C2"], yticklabels=["A1","A2","B1","B2","C1","C2"])

plt.xlabel('true label')
plt.ylabel('predicted label')

# Decision Tree

In [ ]:
classifier_tree = DecisionTreeClassifier(random_state=0)
classifier_tree.fit(x_vect_train, y_train)

In [ ]:
score_tree = classifier_tree.score(x_vect_test, y_test)
print('Accuracy', score_tree)

In [ ]:
y_pred = classifier_tree.predict(x_vect_test)
difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in y_pred]
original_difficulties = [difficulty_levels[label] for label in y_test]

cm = confusion_matrix(original_difficulties, predicted_difficulties, labels=["A1","A2","B1","B2","C1","C2"])
df_cm = pd.DataFrame(cm, index=["A1","A2","B1","B2","C1","C2"], columns= ["A1","A2","B1","B2","C1","C2"])
df_cm

In [ ]:
sb.heatmap(cm.T, square=True, annot=True, fmt='d', cbar=False, xticklabels=["A1","A2","B1","B2","C1","C2"], yticklabels=["A1","A2","B1","B2","C1","C2"])

plt.xlabel('true label')
plt.ylabel('predicted label')

# random forest

In [ ]:
classifier_forest = RandomForestClassifier(random_state=0)
classifier_forest.fit(x_vect_train, y_train)

In [ ]:
score_forest = classifier_forest.score(x_vect_test, y_test)
print('Accuracy', score_forest)

In [ ]:
y_pred = classifier_forest.predict(x_vect_test)

difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in y_pred]
original_difficulties = [difficulty_levels[label] for label in y_test]

cm = confusion_matrix(original_difficulties, predicted_difficulties, labels=["A1","A2","B1","B2","C1","C2"])
df_cm = pd.DataFrame(cm, index=["A1","A2","B1","B2","C1","C2"], columns= ["A1","A2","B1","B2","C1","C2"])
df_cm

In [ ]:
sb.heatmap(cm.T, square=True, annot=True, fmt='d', cbar=False, xticklabels=["A1","A2","B1","B2","C1","C2"], yticklabels=["A1","A2","B1","B2","C1","C2"])

plt.xlabel('true label')
plt.ylabel('predicted label')